In [1]:
import boto3

In [2]:
import os
os.environ.setdefault('AWS_PROFILE', 'gh-activity')

'gh-activity'

In [3]:
dynamodb = boto3.resource('dynamodb')
job_details_table = dynamodb.Table('jobs')

In [4]:
job_details_table.delete_item(Key={'job_id': 'ghactivity_ingest'}) 

{'ResponseMetadata': {'RequestId': '26GJ08DU5J156RGGOKGO3KRN7VVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 16 Nov 2022 11:38:44 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '26GJ08DU5J156RGGOKGO3KRN7VVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [5]:
job_details = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}

In [6]:
job_details_table.put_item(Item=job_details)

{'ResponseMetadata': {'RequestId': 'E1N80L70E3N4PCJ38A5M641CMVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 16 Nov 2022 11:39:21 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'E1N80L70E3N4PCJ38A5M641CMVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [7]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [8]:
job_details = get_job_details('ghactivity_ingest')
job_details

{'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [9]:
from datetime import datetime as dt
from datetime import timedelta as td

In [10]:
dt.now()

datetime.datetime(2022, 11, 16, 17, 33, 30, 780397)

In [11]:
dt.now().date()

datetime.date(2022, 11, 16)

In [12]:
dt.now().date() - td(days=3)

datetime.date(2022, 11, 13)

In [13]:
f'{dt.strftime(dt.now().date() - td(days=3), "%Y-%m-%d")}-0.json.gz'

'2022-11-13-0.json.gz'

In [14]:
next_file = f'{dt.strftime(dt.now().date() - td(days=3), "%Y-%m-%d")}-0.json.gz'

In [15]:
next_file

'2022-11-13-0.json.gz'

In [16]:
import requests

In [17]:
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [18]:
file = open(f'data/{next_file}', 'wb')

In [19]:
file.write(res.content)

47534740

In [20]:
file.close()

In [21]:
!ls -ltr data/

total 79620
-rw-r--r-- 1 edward edward 33989636 Nov 14 17:17 2022-06-05-0.json.gz
-rw-r--r-- 1 edward edward 47534740 Nov 16 17:34 2022-11-13-0.json.gz


In [22]:
import pandas as pd

In [23]:
df = pd.read_json(
    f'data/{next_file}', 
    lines=True, 
    orient='records'
)

In [24]:
df

,id,type,actor,repo,payload,public,created_at,org
0,25199534176,PushEvent,"{'id': 85200428, 'login': 'nicofcabrera', 'dis...","{'id': 546313849, 'name': 'nicofcabrera/app_cl...","{'push_id': 11642771795, 'size': 1, 'distinct_...",True,2022-11-13 00:00:00+00:00,NaN
1,25199534178,PushEvent,"{'id': 116523511, 'login': 'Hazzard1912', 'dis...","{'id': 559373661, 'name': 'Hazzard1912/minipro...","{'push_id': 11642771790, 'size': 1, 'distinct_...",True,2022-11-13 00:00:00+00:00,NaN
2,25199534180,PushEvent,"{'id': 2299951, 'login': 'pietroalbini', 'disp...","{'id': 26193547, 'name': 'rust-lang/crates.io-...","{'push_id': 11642771796, 'size': 1, 'distinct_...",True,2022-11-13 00:00:00+00:00,"{'id': 5430905, 'login': 'rust-lang', 'gravata..."
3,25199534184,PushEvent,"{'id': 56834210, 'login': 'DaemonSharps', 'dis...","{'id': 565182626, 'name': 'DaemonSharps/CI-CD'...","{'push_id': 11642771802, 'size': 1, 'distinct_...",True,2022-11-13 00:00:00+00:00,NaN
4,25199534186,PullRequestEvent,"{'id': 49699333, 'login': 'dependabot[bot]', '...","{'id': 233656732, 'name': 'crlDB/25JaarHuwelij...","{'action': 'opened', 'number': 20, 'pull_reque...",True,2022-11-13 00:00:00+00:00,NaN
...,...,...,...,...,...,...,...,...
110385,25199847442,PushEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 511721826, 'name': 'detain/scoop-emulat...","{'push_id': 11642985749, 'size': 5, 'distinct_...",True,2022-11-13 00:59:59+00:00,NaN
110386,25199847444,PushEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 372127812, 'name': 'minitu/starter-acad...","{'push_id': 11642985744, 'size': 1, 'distinct_...",True,2022-11-13 00:59:59+00:00,NaN
110387,25199847448,PushEvent,"{'id': 58404352, 'login': 'rmcenlly', 'display...","{'id': 559437057, 'name': 'rmcenlly/Jamtacular...","{'push_id': 11642985753, 'size': 1, 'distinct_...",True,2022-11-13 00:59:59+00:00,NaN
110388,25199847449,PullRequestEvent,"{'id': 106833, 'login': 'mugijiru', 'display_l...","{'id': 182490806, 'name': 'mugijiru/.emacs.d',...","{'action': 'opened', 'number': 1323, 'pull_req...",True,2022-11-13 00:59:59+00:00,NaN


In [25]:
df.count()

id            110390
type          110390
actor         110390
repo          110390
payload       110390
public        110390
created_at    110390
org            25325
dtype: int64

In [26]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')

In [27]:
df.dtypes

id                          int64
type                       object
actor                      object
repo                       object
payload                    object
public                       bool
created_at    datetime64[ns, UTC]
org                        object
dtype: object